# Devlog 2023-07-13

Now that we have done a compatibility pass with all of the system modules, let's do a compatibility test. The results of this test should comport with [the analysis in this spreadsheet](https://docs.google.com/spreadsheets/d/1FF_ovXYQUb65JagcLj8QCmCzs2_Pzd9qnZ5Cr28v4jw/edit?usp=sharing).

Generate the cross product of all available modules and test them one-by-one to see if they run successfully. We'll just run each simulation for a few days and verify that this doesn't throw any exceptions. Ideally, these results will line up with our expected compatibility matrix.

In [1]:
from datetime import date
import time
from epymorph.data import mm_library, geo_library, ipm_library
from itertools import product
import pandas as pd

from epymorph.simulation import Simulation

geo_order = ['pei', 'single_pop', 'us_counties_2015',
             'us_states_2015', 'maricopa_cbg_2019']
mm_order = ['pei', 'sparsemod', 'icecube', 'no']
ipm_order = ['pei', 'simple_sirs', 'sirh', 'no']

# geo_order = ['pei', 'single_pop']
# mm_order = ['pei', 'sparsemod']
# ipm_order = ['pei', 'simple_sirs']

combos = pd.DataFrame(
    product(geo_order, mm_order, ipm_order),
    columns=['geo', 'mm', 'ipm']
)

# Params to satisfy every combo!
params = {
    'alpha': [0.05, 0.4125, 0.5875],
    'beta': 0.4,
    'gamma': 0.1000022,
    'phi': 40,
    'theta': 0.1,
    'move_control': 0.9,
    'infection_duration': 4.0,
    'immunity_duration': 90.0,
    'hospitalization_duration': 14.0,
    'hospitalization_rate': 0.1,
    'infection_seed_loc': 0,
    'infection_seed_size': 100
}


def test(geo, mm, ipm):
    success = True
    t0 = time.perf_counter()
    try:
        Simulation(
            geo=geo_library[geo](),
            mvm_builder=mm_library[mm](),
            ipm_builder=ipm_library[ipm]()
        ).run(params, date(2010, 1, 1), 3)
    except:
        success = False
    t1 = time.perf_counter()
    print(f"{'succeeded' if success else 'failed'}: {geo, mm, ipm} in {((t1 - t0) * 1000.0):.3f} ms")
    return pd.Series([success, t1 - t0])


compat = combos.copy()
compat[['runs', 'runtime']] = combos.apply(
    lambda x: test(x['geo'], x['mm'], x['ipm']),
    axis=1)


succeeded: ('pei', 'pei', 'pei') in 16.733 ms
succeeded: ('pei', 'pei', 'simple_sirs') in 13.957 ms
succeeded: ('pei', 'pei', 'sirh') in 15.194 ms
succeeded: ('pei', 'pei', 'no') in 10.771 ms
succeeded: ('pei', 'sparsemod', 'pei') in 12.086 ms
succeeded: ('pei', 'sparsemod', 'simple_sirs') in 11.168 ms
succeeded: ('pei', 'sparsemod', 'sirh') in 11.940 ms
succeeded: ('pei', 'sparsemod', 'no') in 7.001 ms
succeeded: ('pei', 'icecube', 'pei') in 6.751 ms
succeeded: ('pei', 'icecube', 'simple_sirs') in 6.352 ms
succeeded: ('pei', 'icecube', 'sirh') in 7.539 ms
succeeded: ('pei', 'icecube', 'no') in 3.509 ms
succeeded: ('pei', 'no', 'pei') in 3.777 ms
succeeded: ('pei', 'no', 'simple_sirs') in 3.626 ms
succeeded: ('pei', 'no', 'sirh') in 3.902 ms
succeeded: ('pei', 'no', 'no') in 2.409 ms
failed: ('single_pop', 'pei', 'pei') in 4.117 ms
succeeded: ('single_pop', 'pei', 'simple_sirs') in 5.354 ms
succeeded: ('single_pop', 'pei', 'sirh') in 5.363 ms
succeeded: ('single_pop', 'pei', 'no') in 4

In [2]:
from functools import partial, reduce

boolean_or = partial(reduce, bool.__or__)


mm_compat = compat.groupby(by=['geo', 'mm'])\
    .agg({'runs': boolean_or})\
    .reset_index()\
    .pivot(index='geo', columns='mm', values='runs')\
    .reset_index()\
    .reindex(columns=['geo', *mm_order])\
    .sort_values(by='geo', key=lambda x: x.map({k: v for v, k in enumerate(geo_order)}))

display(mm_compat)

ipm_compat = compat.groupby(by=['geo', 'ipm'])\
    .agg({'runs': boolean_or})\
    .reset_index()\
    .pivot(index='geo', columns='ipm', values='runs')\
    .reset_index()\
    .reindex(columns=['geo', *ipm_order])\
    .sort_values(by='geo', key=lambda x: x.map({k: v for v, k in enumerate(geo_order)}))

display(ipm_compat)

# NOTE: and when I ran this, it matched my expectations as laid out in this spreadsheet:
# https://docs.google.com/spreadsheets/d/1FF_ovXYQUb65JagcLj8QCmCzs2_Pzd9qnZ5Cr28v4jw/edit?usp=sharing


mm,geo,pei,sparsemod,icecube,no
1,pei,True,True,True,True
2,single_pop,True,True,True,True
3,us_counties_2015,True,True,True,True
4,us_states_2015,True,True,True,True
0,maricopa_cbg_2019,False,False,True,True


ipm,geo,pei,simple_sirs,sirh,no
1,pei,True,True,True,True
2,single_pop,False,True,True,True
3,us_counties_2015,False,True,True,True
4,us_states_2015,False,True,True,True
0,maricopa_cbg_2019,False,True,True,True


In [3]:
# What are the slowest sims (that succeed)?
compat\
    .loc[compat['runs'] == True]\
    .sort_values(by='runtime', ascending=False)\
    .reset_index(drop=True)


,geo,mm,ipm,runs,runtime
0,us_counties_2015,pei,sirh,True,27.654117
1,us_counties_2015,pei,simple_sirs,True,25.521008
2,us_counties_2015,pei,no,True,21.511111
3,us_counties_2015,sparsemod,sirh,True,18.149128
4,us_counties_2015,sparsemod,simple_sirs,True,16.913561
5,us_counties_2015,sparsemod,no,True,13.081147
6,us_counties_2015,icecube,sirh,True,4.677885
7,us_counties_2015,icecube,simple_sirs,True,4.231097
8,us_counties_2015,icecube,no,True,2.664622
9,maricopa_cbg_2019,icecube,sirh,True,1.657126
